##### <font color='darkblue'>Week 8 - Lab Notebook - Part 1 - Working with Ontologies & Functional Enrichment Analysis</font>

- November 2023
- https://https://github.com/tisimpson/bioinformatics1
- ian.simpson@ed.ac.uk

##### <font color='darkblue'>Introduction</font>
In this notebook we’re going to be doing some manipulation and work with ontologies and then execute a functional enrichment analysis as we discussed in last week's lecture.

##### <font color='darkblue'>Learning Outcomes</font>
After this notebook you should be comfortable with:
- loading an ontology file and some basic searching
- Automating processes using APIs and basic usage of the GSEAPy python package

##### <font color='darkblue'>Useful References</font>
- [GSEAPy Documentation](https://gseapy.readthedocs.io/en/latest/index.html)
- [KEGG API](https://www.kegg.jp/kegg/rest/keggapi.html)
- [Pronto Library](https://pronto.readthedocs.io/en/stable/)

##### <font color='darkblue'>Setting up the Programming Environment</font>

In [ ]:
# You may need to install these packages first
# %pip install gseapy
# %pip install pronto


# handling www based requests (like APIs)
import urllib as ul

# standard Python data handling modules
import pandas as pd
import numpy as np
# working with geen set enrichment analysis (GSEA)
import gseapy
# working with nicer tables
import prettytable as PrettyTable

##### <font color='darkblue'>Using Pronto to Parse and Explore Ontologies</font>

In [ ]:
# fetch the Gene Onotlogy (GO) OBO file and parse it with pronto

# download the GO ontology OBO file
import urllib.request

# download the file
urllib.request.urlretrieve('http://purl.obolibrary.org/obo/go.obo','../data/ontology/go.obo')

# use pronto to parse the file and search for matches in the column moi_curie
import pronto

# parse the file
go = pronto.Ontology('../data/ontology/go.obo')

In [ ]:
# if we know the GO term, we can search for it
goTerm = go['GO:0000002']

# it has features GOTerm Accession, Name (Description), and the clade in GO
print(goTerm.id, goTerm.name, goTerm.namespace)

In [ ]:
# given a string of GO terms, we can search for them by accession in a simple loop

# create a list of GO terms
goTerms = ['GO:0000002', 'GO:0000003', 'GO:0000005', 'GO:0000006', 'GO:0000007']

# loop through the list and print the GO term name and namespace
for goTerm in goTerms:
    print(f"",go[goTerm].name," clade :",go[goTerm].namespace)

In [ ]:
# we can also search for GO terms by name
# create a list of GO term names
goNames = ['mitochondrial genome maintenance', 'developmental process', 'apoptotic process', 'reproduction', 'cell death']

# loop through the go object and compare the strings in goNames to the name attribute of the GO term
for goTerm in go:
    try:
        currentTerm = go.get_term(goTerm)
        if currentTerm.name in goNames:
            print(f"{currentTerm.id} {currentTerm.name} {currentTerm.namespace}")
    except:
        pass

# it's a little clunky (pronto wasn't specifically designed for this), but it works well enough

In [ ]:
# we can explore the tree strucutre of the onology using pronto

# picking a starting GO term we can find all of the child terms underneath it
# here we start with GO:0032502 (developmental process)
# we can use the children() method to get all of the child terms

# get the GO term children
goTerms = go['GO:0032502'].subclasses().to_set()

# how many terms are there?
print(len(goTerms))

# print the GO term children
for goTerm in goTerms:
    print(f"{goTerm.id} {goTerm.name}")

##### <font color='darkblue'>Download KEGG Pathway Information</font>

In [ ]:
# First we fetch the list of human pathways available at KEGG.

human_pathways = pd.read_csv(ul.request.urlopen('http://rest.kegg.jp/list/pathway/hsa'),sep='\t',header=0,names=['kegg_id','pathway_name'])
human_pathways.head()

# We specifically want the pathway data for the "Dopaminergic Synapse" pathway.
pathway_info = human_pathways[human_pathways['pathway_name'].str.match('Dopaminergic synapse')]['kegg_id']

# extract the exact pathway accession
pathway_id = pathway_info.values[0].split('\t')[0]

# pull the pathway directly from KEGG, note we are saving this to a file 'dop_synapse.txt' that we will use later
ul.request.urlretrieve('http://rest.kegg.jp/get/'+pathway_id,'../data/pathways/dop_synapse.txt');

In [ ]:
# Now we will use this file which contains the full pathway details including the gene names.

# open the file
dop_file = open('../data/pathways/dop_synapse.txt','r')

# I wanted to show you some basic python parsing and a simple for loop with a conditional in to demonstrate how you can quickly build simple parsers.
# There are quicker ways to do this, but this is a good learning example.

# create an empty dataframe with two columns
dop_df = pd.DataFrame(columns=['gene_id','gene_symbol','description'])

# set a flag for our parser
flag=0

# work through the text file one line at a time
for line in dop_file:
    # find the start of the gene entries
    if 'GENE' in line:
        # add the first gene tp the dataframe
        gene_id,remain = line.strip('GENE').strip().split('  ')
        gene_symbol,description = remain.split(';')
        # add a new row to the dataframe containing the gene_id and description
        dop_df = pd.concat([dop_df,pd.DataFrame([[gene_id,gene_symbol,description]],columns=['gene_id','gene_symbol','description'])],ignore_index=True)
        # set the flag to 1, we are in the gene section of the file
        flag = 1
    # stop when we reach the end of the section and escape the file
    elif 'COMPOUND' in line:
        break
    # continue adding the genes to the dataframe
    elif flag == 1:
        gene_id,remain = line.strip('GENE').strip().split('  ')
        gene_symbol,description = remain.split(';')
        # add the gene to the dataframe
        dop_df = pd.concat([dop_df,pd.DataFrame([[gene_id,gene_symbol,description]],columns=['gene_id','gene_symbol','description'])],ignore_index=True)

# close the file
dop_file.close()

# view the file
dop_df.head()

# you now have the gene_ids (NCBI EntrezIDs for the genes in the pathway)
print('The Dopaminergic Synapse pathway has '+str(dop_df.shape[0])+' genes in it.\n')

# store the gene_ids in a numpy array
gene_ids = dop_df['gene_id'].to_numpy()

gene_symbols = dop_df['gene_symbol'].to_numpy()

# show the gene_ids
print(gene_ids)

# write the gene_ids to a file
with open('../data/pathways/dop_geneids.txt','w') as f:
    for gene_id in gene_ids:
        f.write(gene_id+'\n')

# show the gene_symbols
print(gene_symbols)

# write the gene_symbols to a file
with open('../data/pathways/dop_symbols.txt','w') as f:
    for symbol in gene_symbols:
        f.write(symbol+'\n')

##### <font color='darkblue'>GSEAPy and Gene Set Enrichement Analysis</font>

[GSEApy](https://gseapy.readthedocs.io/en/latest/index.html) is a library to perform gene set enrichment analysis (GSEA) in python. There are two methods to perform enrichment analysis - over representation analysis and GSEA. The main difference between the two is that GSEA assumes your input list of genes is ordered by the most representative genes in that list. 

In this step we use biomart which is an excellent service run at EBI-Ensembl that allows you to query and retrieve linked data for genomic data.

The help for Biomart can be found here - [https://www.ensembl.org/info/data/biomart/how_to_use_biomart.html](https://www.ensembl.org/info/data/biomart/how_to_use_biomart.html)

Biomart API functionality is nicely delivered through GSEApy and its use is described here - [https://gseapy.readthedocs.io/en/latest/gseapy_example.html#1.-Biomart-API](https://gseapy.readthedocs.io/en/latest/gseapy_example.html#1.-Biomart-API).

In [ ]:
# the GSEApy package also contains functions that allow you to use the EBI-Ensembl Biomart service
# you can use this to directly query linked data for the genes, including Gene Ontology (GO) annotation data.
from gseapy import Biomart

# initiate a biomart connection
bm = Biomart()

# form a query for biomart from the Entrez Gene IDs, here we are using the gene_ids from the dopaminergic synapse pathway above
queries = {'entrezgene_id' : gene_ids}

# execute the biomart query
# NB that the oddly named 'name_1006' attribute is in fact the 'GO Term name' attribute
# NB a nice trick for finding the correct attribute names is to use the website to create the query and then
# NB click on the XML link at the top of the page, this will show you the 'Attribute name'.
results = bm.query(dataset='hsapiens_gene_ensembl',
                   attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id', 'go_id','go_linkage_type','name_1006'],
                   filters=queries)

# change the name to something more useful
results.rename({'name_1006' : 'go_name'},axis=1,inplace=True)

results.head()

In [ ]:
# plot the top20 most common GO terms from the results table using seaborn
# seaborn is a Python NB you need seaborn v 0.13+ for this to work
import seaborn as sns
import matplotlib.pyplot as plt

# plot the top20 most common GO terms
sns.countplot(y='go_name',data=results,order=results['go_name'].value_counts().iloc[:20].index,palette='bright',hue='go_name',legend=False)
# add a title
plt.title('Top 20 GO Terms for Dopaminergic Synapse Pathway')
# add a label to the x axis
plt.xlabel('Gene Count')
# add a label to the y axis
plt.ylabel('GO Term Name')
# show the plot
plt.show()

In [ ]:
# just to take extra advantage of the data lets break down the above by evidence code type
annotations_by_evidence = pd.DataFrame(results[['go_name','go_linkage_type']].dropna())

# we use the Pandas pivot_table function to do some powerful refactoring - count and sort the pairs go_term & evidence code
annotations_by_evidence = annotations_by_evidence.pivot_table(index=['go_name','go_linkage_type'],aggfunc='size').reset_index(name='frequency').sort_values(by='frequency',ascending=False)

# just take the top 30 rows
top_annotations = annotations_by_evidence[:30]

# pivot to create a nice matrix for a stacked plot
top_annotations = top_annotations.pivot(index='go_name',columns='go_linkage_type',values='frequency').fillna(0)

# create a staked bar plot using seaborn
top_annotations.plot.barh(xlabel='Number of Entries',
    ylabel='GO Term Name',
    title='Top Frequent GO Annotations by Evidence Code in the Dopaminergic Synapse Pathway',
    stacked=True).legend(loc='best');

##### <font color='darkblue'>Enrichment Analysis Measures</font>

In order to perform GSEA we need to impose an ordering on the lists of elements (such as genes) and compare the differential position of those in lists separated by two classes as we discussed in the lecture. In the next notebook we will do this with some network derived data, but here we are going to perform the simpler variant (which is still useful) over representation analysis (ORA) as (at the moment) we don't have suitable lists for GSEA.

You can browse the available gene sets to perform enrichment analysis against using the GSEAPy package at the [Enrichr website](https://maayanlab.cloud/Enrichr/#libraries).

In [ ]:
# this is incredibly easy to do with the GSEApy package
# we provide a list of gene_symbols and select the pathway/list to compare against
# note that by default this is comparing against the entire genome annotation
# in a real experimental set up we need to define the foreground and background gene sets as discussed in the lecture

enr = gseapy.enrichr(gene_list=list(gene_symbols), # perform enrichment analysis using gsea
                 gene_sets=['KEGG_2021_Human'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )

enr.results.head(10) #return the top 10 hits

In [ ]:
# we can list the available gene sets
available_libraries = gseapy.get_library_name()

# how many are there?
print(f'There are ',len(available_libraries),' available gene sets to compare against!')

# print the first 10 using prettytable
from prettytable import PrettyTable
table = PrettyTable(['Library Name'])
for library in available_libraries[:10]:
    table.add_row([library])
print(table)


In [ ]:
# perform ORA against the Hallmark gene sets
enr = gseapy.enrichr(gene_list=list(gene_symbols), # perform enrichment analysis using gsea
                 gene_sets=['MSigDB_Hallmark_2020'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )

enr.results.head(10) #return the top 10 hits